## BLUR CANNY CONTOUR 

In [ ]:
from PIL import Image
import pandas as pd
import os

In [ ]:
anot_path = "./data/attention.csv"
images_path = "./data/datasets/"

gray_images_path = "./data/grayimage"

In [ ]:
df = pd.read_csv(anot_path)
df['score'].value_counts()

In [ ]:
save_path = "gray_data.csv"

data = {
    'width': [],
    'heigh': [],
    'file_path': [],
    'score': []
}

if not os.path.exists(gray_images_path):
    os.makedirs(gray_images_path)

for item, score in zip(df['filename'].tolist(), df['score'].tolist()):
    file_path = os.path.join(images_path, item)
    gray_path = os.path.join(gray_images_path, item)

    img = Image.open(file_path).convert("L")
    
    img.save(gray_path)
    
    data['width'].append(img.size[0])
    data['heigh'].append(img.size[1])
    data['file_path'].append(gray_path)
    data['score'].append(score)

In [ ]:
import pandas as pd
from PIL import Image
import random
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

rd_idx = random.randint(0, 99)
# anomali data 69, 5, 36
# rd_idx = 5

file_path = "gray_data.csv"
df = pd.read_csv(file_path)

sample_image_name = df['file_path'].tolist()[rd_idx]
score = df['score'].tolist()[rd_idx]

print(f"image idx: {rd_idx} | score: {score}")

img_np = np.array(Image.open(sample_image_name))

blank = np.zeros(shape=img_np.shape, dtype='uint8')

gaussBlur = cv.GaussianBlur(img_np, (175, 175), 0.3)
canny = cv.Canny(gaussBlur, 100, 100)

_, tresh = cv.threshold(img_np, 125, 255, cv.THRESH_BINARY)
_, blurtresh = cv.threshold(gaussBlur, 125, 255, cv.THRESH_BINARY)

contours, _ = cv.findContours(canny, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
cv.drawContours(blank, contours, -1, 255, 1)

In [ ]:
images = [img_np, gaussBlur, canny, tresh, blank]
titles = ['Original', 'Gaussian Blur', 'Canny', 'Threshold', 'Contours Drawn']

plt.figure(figsize=(15, 8))

for i in range(len(images)):
    plt.subplot(2, 3, i + 1)  # 2 baris, 3 kolom
    if len(images[i].shape) == 2:  # grayscale
        plt.imshow(images[i], cmap='gray')
    else:  # RGB image
        plt.imshow(cv.cvtColor(images[i], cv.COLOR_BGR2RGB))
    plt.title(titles[i])
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
max_w = max(df['width'].tolist())
min_w = min(df['width'].tolist())

max_h = max(df['height'].tolist())
min_h = min(df['height'].tolist())

print(f"max-width: {max_w} | min-width: {min_w} | max-height: {max_h} | min-height: {min_h}")